# IndicWav2Vec (Hindi) Speech Recognition

In [ ]:
!pip install torch torchaudio transformers pydub

In [ ]:

import torch
from pydub import AudioSegment
import torchaudio
from transformers import AutoModelForCTC, AutoProcessor

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_ID = "ai4bharat/indicwav2vec-hindi"
MP3_PATH = "download.mp3"

# Load MP3 file
audio = AudioSegment.from_file(MP3_PATH)
audio.export("audio.wav", format="wav")

# Load model and processor
processor = AutoProcessor.from_pretrained(MODEL_ID)
model = AutoModelForCTC.from_pretrained(MODEL_ID).to(DEVICE)

# Load audio
waveform, rate = torchaudio.load("audio.wav")
inputs = processor(waveform.squeeze().numpy(), sampling_rate=rate, return_tensors="pt", padding=True)

# Inference
with torch.no_grad():
    logits = model(inputs.input_values.to(DEVICE)).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
print("Transcription:", transcription[0])
